In [1]:
import numpy as np
import pandas as pd
import time

I have invented closed form retrieving the state price (Arrow-Debreu security price) matrix from spot state price vectors observed.

# 2. Algorithm for Basic Ross 

In [2]:
st = time.time()
np.random.seed(2025)

n = 120 # the number of possible states and tenors
r = 5 # the initial state is w_6

print(f"n (# of time points observed): {n}")
print(f"N (# of possible states): {n}")
print("-" * 100)
print(f"Initial state: w{r+1}")

# Suppose that we have observed spot state prices
temp = []
for i in range(n):
    pi_spot = np.random.rand(n)
    temp.append(pi_spot)
spot_mat = np.vstack(temp)

K = np.delete(np.delete(spot_mat, r, axis=1), n - 1, axis=0)
b = np.concatenate([spot_mat[i] - spot_mat[i - 1][r] * spot_mat[0] for i in range(1, n)])
M = np.kron(K, np.eye(n))
vec_Qp = np.kron(np.linalg.inv(K), np.eye(n)) @ b
Qp = vec_Qp.reshape(n, n-1, order='F')
Q = np.insert(Qp, r, spot_mat[0], axis=1)
Pi = Q.T

for i in range(1, n):
    pi_spot = spot_mat[i]
    pi_spot_hat = Q @ spot_mat[i - 1] # induction
    if not np.allclose(pi_spot, pi_spot_hat):
        print("Test Failed:")
        print("Estimated Value:", pi_spot_hat)
        print("Initial Value:", pi_spot)
        break
        
print("-" * 100) 
print("All Tests Passed")
print("Time Required:", round(time.time() - st, 4), "seconds")
print("Pi:", Pi.shape)
print("-" * 100)

n (# of time points observed): 120
N (# of possible states): 120
----------------------------------------------------------------------------------------------------
Initial state: w6
----------------------------------------------------------------------------------------------------
All Tests Passed
Time Required: 1.233 seconds
Pi: (120, 120)
----------------------------------------------------------------------------------------------------


# 3. Algorithm for Overlapping Method (Audrino et al (2020))

In [3]:
st = time.time()
np.random.seed(2025)

n = 36 # the number of tenors
k = 4 # the number of steps in a month
print(f"n (# of months): {n} ({round(n/12, 2)} years)")
print(f"k (# of steps in a month): {k}")
print(f"n * k (# of time points observed): {n * k}")
N = (n - 1) * k + 1 # the number of possible states
print(f"N (# of possible states): {N}")
print("-" * 100)
r = 4 # randomly select the initial state as w_5
print(f"Initial state: w{r+1}")

# Suppose that we have observed spot state prices
temp = []
for i in range(n * k):
    pi_spot = np.random.rand(N)
    temp.append(pi_spot)
spot_mat = np.vstack(temp)
K = np.delete(spot_mat[:(N-1), :], r, axis=1)
b = np.concatenate([
    spot_mat[k + i] - spot_mat[k - 1] * spot_mat[i][r] for i in range(N - 1)
])
M = np.kron(K, np.eye(N))
vec_Qp = np.linalg.solve(M, b)
Qp = vec_Qp.reshape(N, N-1, order='F')
Q = np.insert(Qp, r, spot_mat[k - 1], axis=1)
Pi = Q.T

for i in range(N - 1):
    pi_spot = spot_mat[k + i]
    pi_spot_hat = Q @ spot_mat[i] # induction
    if not np.allclose(pi_spot, pi_spot_hat):
        print(f"Test Failed at {i}:")
        print("Estimated Value:", pi_spot_hat)
        print("Initial Value:", pi_spot)
        break
        
print("-" * 100)
print("All Tests Passed")
print("Time Required:", round(time.time() - st, 4), "seconds")
print("Pi:", Pi.shape)
print("-" * 100)

n (# of months): 36 (3.0 years)
k (# of steps in a month): 4
n * k (# of time points observed): 144
N (# of possible states): 141
----------------------------------------------------------------------------------------------------
Initial state: w5
----------------------------------------------------------------------------------------------------
All Tests Passed
Time Required: 225.2351 seconds
Pi: (141, 141)
----------------------------------------------------------------------------------------------------


# 4. Acceleration of Overlapping Method

In [4]:
st = time.time()
np.random.seed(2025)

n = 36 # the number of tenors
k = 4 # the number of steps in a month
print(f"n (# of months): {n} ({round(n/12, 2)} years)")
print(f"k (# of steps in a month): {k}")
print(f"n * k (# of time points observed): {n * k}")
N = (n - 1) * k + 1 # the number of possible states
print(f"N (# of possible states): {N}")
print("-" * 100)
r = 4 # randomly select the initial state as w_5
print(f"Initial state: w{r+1}")

# Suppose that we have observed spot state prices
temp = []
for i in range(n * k):
    pi_spot = np.random.rand(N)
    temp.append(pi_spot)
spot_mat = np.vstack(temp)
K = np.delete(spot_mat[:(N-1), :], r, axis=1)

# key changes
B = np.vstack([
    spot_mat[k + i] - spot_mat[k - 1] * spot_mat[i][r]
    for i in range(N - 1)
])
Qp = np.linalg.solve(K, B).T

Q = np.insert(Qp, r, spot_mat[k - 1], axis=1)
Pi = Q.T

for i in range(N - 1):
    pi_spot = spot_mat[k + i]
    pi_spot_hat = Q @ spot_mat[i] # induction
    if not np.allclose(pi_spot, pi_spot_hat):
        print(f"Test Failed at {i}:")
        print("Estimated Value:", pi_spot_hat)
        print("Initial Value:", pi_spot)
        break
        
print("-" * 50)
print("All Tests Passed")
print("Time Required:", round(time.time() - st, 4), "seconds")
print("Pi:", Pi.shape)
print("-" * 50)

n (# of months): 36 (3.0 years)
k (# of steps in a month): 4
n * k (# of time points observed): 144
N (# of possible states): 141
----------------------------------------------------------------------------------------------------
Initial state: w5
--------------------------------------------------
All Tests Passed
Time Required: 0.0087 seconds
Pi: (141, 141)
--------------------------------------------------


# 5. Concrete Example

In [5]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

n = 60 # the number of tenors
k = 4 # the number of steps in a month
print(f"n (# of months): {n} ({round(n/12, 2)} years)")
print(f"k (# of steps in a month): {k} (weekly)")
print(f"n * k (# of time points observed): {n * k}")
N = (n - 1) * k + 1 # the number of possible states
print(f"N (# of possible states): {N}")

print("-" * 100)
print(f"We can partition the total state space into a granular grid of {N} points")
sigma_range = np.linspace(-5, 5, N)
print(f"i.e.) [{round(sigma_range[0],3)}σ, {round(sigma_range[1],3)}σ, {round(sigma_range[2],3)}σ, ... , {round(sigma_range[-3],3)}σ, {round(sigma_range[-2],3)}σ, {round(sigma_range[-1],3)}σ]")


print("-" * 100)
r = 99 # randomly select the initial state as w_100
print(f"Assume that the initial state is w_{r+1} = {round(-5 + 10/N * (r+1), 3)}σ")
print("-" * 100)

base_date = datetime(2025, 6, 1)
print("Suppose that we observed weekly spot state prices from SPX European option prices as:")
temp = []
for i in range(n * k):
    pi_spot = np.random.rand(N)
    temp.append(pi_spot)
spot_mat = np.vstack(temp)

for i in [0, 1, -1, n*k-2, n*k-1]:
    if i == -1:
        print("...")
        continue
    next_week_date = base_date + timedelta(weeks=i)
    lst = [round(x, 2) for x in temp[i]]
    print(f"(week {i+1}) {next_week_date.strftime("SPXW %m/%d/%y")}:", [lst[0], lst[1], "...", lst[-2], lst[-1]])

K = np.delete(spot_mat[:(N-1), :], r, axis=1)

# key changes
B = np.vstack([
    spot_mat[k + i] - spot_mat[k - 1] * spot_mat[i][r]
    for i in range(N - 1)
])
Qp = np.linalg.solve(K, B).T

Q = np.insert(Qp, r, spot_mat[k - 1], axis=1)
Pi = Q.T

for i in range(N - 1):
    pi_spot = spot_mat[k + i]
    pi_spot_hat = Q @ spot_mat[i] # induction
    if not np.allclose(pi_spot, pi_spot_hat):
        print(f"Test Failed at {i}:")
        print("Estimated Value:", pi_spot_hat)
        print("Initial Value:", pi_spot)
        break

print("-" * 100)
print("The transition state price matrix recovered from the spot state prices is:")
pd.set_option('display.max_columns',9)
pd.set_option('display.float_format', lambda x: f"{x:.3f}")
labels = [f"{val:.3f}σ" for val in sigma_range]
display(pd.DataFrame(Pi, index=labels, columns=labels))

n (# of months): 60 (5.0 years)
k (# of steps in a month): 4 (weekly)
n * k (# of time points observed): 240
N (# of possible states): 237
----------------------------------------------------------------------------------------------------
We can partition the total state space into a granular grid of 237 points
i.e.) [-5.0σ, -4.958σ, -4.915σ, ... , 4.915σ, 4.958σ, 5.0σ]
----------------------------------------------------------------------------------------------------
Assume that the initial state is w_100 = -0.781σ
----------------------------------------------------------------------------------------------------
Suppose that we observed weekly spot state prices from SPX European option prices as:
(week 1) SPXW 06/01/25: [np.float64(0.13), np.float64(0.0), '...', np.float64(0.73), np.float64(0.93)]
(week 2) SPXW 06/08/25: [np.float64(0.75), np.float64(0.65), '...', np.float64(0.01), np.float64(0.2)]
...
(week 239) SPXW 12/23/29: [np.float64(0.93), np.float64(0.14), '...', np.float6

,-5.000σ,-4.958σ,-4.915σ,-4.873σ,...,4.873σ,4.915σ,4.958σ,5.000σ
-5.000σ,-0.796,0.316,-0.023,0.289,...,-0.317,0.169,0.048,0.742
-4.958σ,-2.524,1.916,-0.408,1.049,...,-1.721,0.810,0.232,1.299
-4.915σ,2.412,-1.636,0.682,-1.118,...,1.527,-0.688,-0.599,-0.846
-4.873σ,1.558,-2.393,-0.767,2.092,...,0.591,0.184,-0.322,-0.348
-4.831σ,0.213,-0.249,0.247,-0.297,...,0.932,-0.222,-0.291,-1.195
...,...,...,...,...,...,...,...,...,...
4.831σ,0.723,-1.690,-0.219,-0.353,...,1.024,-0.350,-0.611,-1.272
4.873σ,1.046,-0.724,0.731,-1.809,...,1.262,-0.631,-0.656,-2.428
4.915σ,-4.875,5.717,-0.167,-0.554,...,-3.092,0.147,1.054,2.633
4.958σ,1.828,1.027,2.336,-3.424,...,1.390,-1.469,0.536,-1.664
